<a href="https://colab.research.google.com/github/esmaeilifardi/California-housing-price-prediction-with-python/blob/dev/housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv("housing.csv")
df

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

**visualization**

In [15]:
#df.plot(kind='scatter', x='longitude', y='latitude', figsize=(10, 10), alpha=0.2, s=df['population']/25, label='population',
 #c=df['median_house_value'] , cmap=plt.get_cmap('jet') ) 

In [ ]:
df.plot.scatter(x='longitude', y='latitude', figsize=(15, 10), alpha=0.2, s=df['population']/25, label='population',
 c=df['median_house_value'] , cmap=plt.get_cmap('jet') ) 
plt.show()

In [ ]:
df.plot.scatter(x = 'median_income', y = 'median_house_value', figsize = (15, 10))

In [ ]:
scatter_matrix(df, figsize=(20, 15)) 
plt.show()

In [ ]:
corr_matrix = df.corr()
corr_matrix

In [ ]:
print( corr_matrix['median_house_value'].sort_values(ascending=False) ) 

In [ ]:
sns.heatmap(corr_matrix)
plt.show()

In [22]:
#====================================================(prepare data)==================================================

In [23]:
df_label = df['median_house_value'].copy()

In [24]:
df = df.drop('median_house_value', axis = 1)

In [25]:
df_cat = df['ocean_proximity'].copy()

In [26]:
df_num = df.drop('ocean_proximity', axis =1)

**Missing Values**

**Method A)**

In [27]:
#df_num.drop('total_bedrooms', axis = 1)                 #B  hazfe kole sootoon
#df_num = df_num.dropna(subset = ['total_bedrooms'])     #A  hazfe khali ha
#median = df_num['total_bedrooms'].median()              #C  mianeh
#df_num['total_bedrooms'].fillna(median)

**Method B)**

In [28]:
imputer = SimpleImputer(missing_values= np.nan, strategy= 'median' )
X = imputer.fit_transform(df_num)
df_num_impute_tr = pd.DataFrame(X, columns = df_num.columns)



In [ ]:
df_num_impute_tr.isnull().sum()

In [ ]:
df_num_impute_tr.columns

***feature scaling ***

**normalized, standard, minmax, robust**


In [31]:
feature_scal = StandardScaler()
data_num_scaled_tr = feature_scal.fit_transform(df_num_impute_tr)

In [32]:
data_num_scaled_tr = pd.DataFrame(data_num_scaled_tr, columns = df_num_impute_tr.columns)

**label encoding**

In [33]:
#encoder = LabelEncoder()
#data_cat_encoded = encoder.fit_transform(df_cat)
#data_cat_encoded = pd.DataFrame(data_cat_encoded, columns = df_cat.columns)

**one hot encoding**

In [ ]:
encoder_1hot = OneHotEncoder(sparse= False) # آپشن اسپارس برای حالتی است که داده های نال را در نظر نگیرد
df_cat_1hot = np.array(df_cat).reshape((len(df_cat), 1))
df_cat_1hot = encoder_1hot.fit_transform(df_cat_1hot)
df_cat_1hot = pd.DataFrame(df_cat_1hot)
df_cat_1hot.columns = encoder_1hot.get_feature_names(['prox'])
df_cat_1hot

In [35]:
final = pd.concat( [data_num_scaled_tr, df_cat_1hot], axis =1 ) # کوتیشن ندارد

In [ ]:
final

**split data**

In [37]:
X_train, X_test, y_train, y_test = train_test_split(final, df_label, test_size= 0.2, random_state= 11)

**Model** **Selection**

In [38]:
forest_reg = RandomForestRegressor()

In [ ]:
param_grid = [ {'n_estimators': [3, 4, 6, 10, 30], 'max_features': [2, 6, 8, 15] } ]
grid_search = GridSearchCV(forest_reg, param_grid, cv= 5, scoring= 'neg_mean_squared_error') # cv=5 یعنی پنج بار این کار را انجام بده
grid_search.fit(X_train, y_train)

print('best parameter:' , grid_search.best_params_)
print('best estimator:' , grid_search.best_estimator_)

In [40]:
final_model = grid_search.best_estimator_
# X_test = full_pipeline.transform(X2)
final_pricitions = final_model.predict(X_test)
final_mse = mean_squared_error(final_pricitions, y_test)    #(y_predict, y_test)
final_rmse = np.sqrt(final_mse)
print(final_rmse)

47627.74732730439
